# A full business solution - – Enhanced Version


## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

### Bharat Puri - Integrated translator feature on the existing brocher creation process

In [16]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
# To handle path of scraper go up 2 directories
import sys
sys.path.append(os.path.abspath(os.path.join("..", ".."))) 
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [17]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'

openai = OpenAI()

API key looks good so far


In [18]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [19]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [20]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [21]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [14]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [22]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'course page',
   'url': 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/'},
  {'type': 'linkedin page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [23]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [24]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog / insights', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [25]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
PaddlePaddle/PaddleOCR-VL
Updated
1 day ago
•
3.82k
•
609
nanonets/Nanonets-OCR2-3B
Updated
3 days ago
•
12.8k
•
316
Qwen/Qwen3-VL-8B-Instruct
Updated
4 days ago
•
74.5k
•
186
Phr00t/Qwen-Image-Edit-Rapid-AIO
Updated
about 16 hours ago
•
349
inclusionAI/Ling-1T
Updated
5 days ago
•
3k
•
450
Browse 1M+ models
Spaces
Building
294
294
veo3.1-fast
🐨
Generate videos from text or images
Running
15.2k
15.2k
DeepSite v

In [26]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nPaddlePaddle/PaddleOCR-VL\nUpdated\n1 day ago\n•\n3.82k\n•\n611\nnanonets/Nanonets-OCR2-3B\nUpdated\n3 days ago\n•\n12.8k\n•\n317\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n4 days ago\n•\n74.5k\n•\n186\nPhr00t/Qwen-Image-Edit-Rapid-AIO\nUpdated\nabout 17 hours ago\n•\n

In [37]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

def translate_brochure(brochure_content, language="Hindi"):
    system_prompt = f"""
    You are a skilled marketing translator.
    Translate the following brochure text into idiomatic {language}.
    Match the tone and style of native marketing materials.
    Output in Markdown.
    """
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_content}
        ],
    )
    translated = response.choices[0].message.content
    display(Markdown(f"### 🌍 Translated Brochure ({language})\n\n{translated}"))
    return translated

In [38]:
brochure = create_brochure("HuggingFace", "https://huggingface.co")
translate_text(brochure, "Spanish")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face: Building the Future of AI Together

---

## About Hugging Face

Hugging Face is at the heart of the AI revolution, serving as the premier collaboration platform for the machine learning community. We empower machine learning engineers, scientists, and developers worldwide to share, discover, and build open and ethical AI solutions. Our mission is clear: **to democratize quality machine learning — one commit at a time.**

The Hugging Face Hub is a vibrant ecosystem where people can host and collaborate on unlimited models, datasets, and applications, accelerating AI innovation through community and open-source tools.

---

## What We Offer

### Models & Datasets  
- Browse **1 million+ models** including cutting-edge NLP, vision, audio, and multimodal models.  
- Access **250,000+ datasets** curated by the community to fuel your AI projects.  

### Spaces  
- Explore and deploy ML applications with over **400,000+ applications** running in the community.  
- Share your own AI apps effortlessly with our collaborative platform.

### Infrastructure  
- Get started with **fast and easy inference** to integrate AI models in seconds.  
- Enterprise-grade compute solutions and powerful APIs built for team collaboration and scaling AI projects.

---

## Our Community & Culture

- A **fast-growing, diverse, and passionate** community of 200+ team members and over 64,000 followers dedicated to open-source AI.  
- We believe in **open science**, collaboration, and ethical AI development.  
- Hugging Face hosts a rich ecosystem where users can **build portfolios, share their ML work, and grow their expertise**.  

### Core Values  
- **Openness:** Everything we build is designed to be accessible and community-driven.  
- **Collaboration:** We foster an environment where learning and innovation happen through shared efforts.  
- **Ethics:** We are committed to building AI responsibly to benefit humanity.

---

## Customers & Use Cases

Our platform supports a broad spectrum of users, from individual researchers and AI enthusiasts to large enterprises, including:

- Machine learning researchers leveraging open datasets and pre-trained models.  
- Enterprises accelerating AI adoption with custom integration and enterprise solutions.  
- Developers creating next-gen AI applications across text, vision, audio, and video modalities.

---

## Careers at Hugging Face

Join a team that is shaping the future of AI!  
We are looking for passionate engineers, researchers, and innovators who want to:

- Work on **state-of-the-art models and open source AI tools**.  
- Collaborate with a **global community of AI pioneers**.  
- Grow their careers in a **dynamic, inclusive, and mission-driven environment**.

For current job openings and application details, visit our [Jobs Page](https://huggingface.co/jobs).

---

## Get Involved

- Explore models and datasets: https://huggingface.co/models  
- Try AI applications on Spaces: https://huggingface.co/spaces  
- Join the community discussions on [Discord](https://discord.gg/huggingface) and follow us on [Twitter](https://twitter.com/huggingface).  

Together, let's build an open and ethical AI future.

---

**Hugging Face**  
The AI community building the future.  
[https://huggingface.co](https://huggingface.co)  

---

### Brand Colors  
- Yellow: #FFD21E  
- Orange: #FF9D00  
- Gray: #6B7280

---

*Empowering the AI community — one commit at a time.*

### 🌍 Translated Brochure (Spanish)

```markdown
# Hugging Face: Construyendo el Futuro de la IA Juntos

---

## Sobre Hugging Face

Hugging Face está en el corazón de la revolución de la IA, sirviendo como la principal plataforma de colaboración para la comunidad de aprendizaje automático. Empoderamos a ingenieros, científicos y desarrolladores de aprendizaje automático en todo el mundo para compartir, descubrir y construir soluciones de IA abiertas y éticas. Nuestra misión es clara: **democratizar el aprendizaje automático de calidad — un commit a la vez.**

El Hugging Face Hub es un ecosistema vibrante donde las personas pueden alojar y colaborar en modelos ilimitados, conjuntos de datos y aplicaciones, acelerando la innovación en IA a través de la comunidad y herramientas de código abierto.

---

## Lo Que Ofrecemos

### Modelos y Conjuntos de Datos  
- Navega **más de 1 millón de modelos** que incluyen modelos avanzados de PLN, visión, audio y multimodalidad.  
- Accede a **más de 250,000 conjuntos de datos** seleccionados por la comunidad para impulsar tus proyectos de IA.  

### Spaces  
- Explora y despliega aplicaciones de aprendizaje automático con más de **400,000 aplicaciones** funcionando en la comunidad.  
- Comparte tus propias aplicaciones de IA sin esfuerzo con nuestra plataforma colaborativa.

### Infraestructura  
- Comienza con **inferencia rápida y sencilla** para integrar modelos de IA en segundos.  
- Soluciones de computación de nivel empresarial y poderosas API diseñadas para la colaboración en equipo y la escalabilidad de proyectos de IA.

---

## Nuestra Comunidad y Cultura

- Una comunidad **rápida, diversa y apasionada** con más de 200 miembros en el equipo y más de 64,000 seguidores dedicados a la IA de código abierto.  
- Creemos en la **ciencia abierta**, la colaboración y el desarrollo ético de la IA.  
- Hugging Face alberga un ecosistema rico donde los usuarios pueden **construir portafolios, compartir su trabajo de aprendizaje automático y crecer en su experiencia**.  

### Valores Fundamentales  
- **Apertura:** Todo lo que construimos está diseñado para ser accesible y dirigido por la comunidad.  
- **Colaboración:** Fomentamos un ambiente donde el aprendizaje y la innovación ocurren a través del esfuerzo compartido.  
- **Ética:** Estamos comprometidos a construir IA responsablemente para beneficiar a la humanidad.

---

## Clientes y Casos de Uso

Nuestra plataforma soporta un amplio espectro de usuarios, desde investigadores individuales y entusiastas de la IA hasta grandes empresas, incluyendo:

- Investigadores en aprendizaje automático que aprovechan conjuntos de datos abiertos y modelos preentrenados.  
- Empresas que aceleran la adopción de IA con integración personalizada y soluciones empresariales.  
- Desarrolladores creando aplicaciones de IA de próxima generación en modalidades de texto, visión, audio y video.

---

## Carreras en Hugging Face

¡Únete a un equipo que está moldeando el futuro de la IA!  
Buscamos ingenieros, investigadores e innovadores apasionados que quieran:

- Trabajar en **modelos de última generación y herramientas de IA de código abierto**.  
- Colaborar con una **comunidad global de pioneros en IA**.  
- Desarrollar su carrera en un **entorno dinámico, inclusivo y orientado a la misión**.

Para vacantes actuales y detalles de aplicación, visita nuestra [Página de Empleos](https://huggingface.co/jobs).

---

## Participa

- Explora modelos y conjuntos de datos: https://huggingface.co/models  
- Prueba aplicaciones de IA en Spaces: https://huggingface.co/spaces  
- Únete a las discusiones de la comunidad en [Discord](https://discord.gg/huggingface) y síguenos en [Twitter](https://twitter.com/huggingface).  

Juntos, construyamos un futuro abierto y ético para la IA.

---

**Hugging Face**  
La comunidad de IA que construye el futuro.  
[https://huggingface.co](https://huggingface.co)  

---

### Colores de Marca  
- Amarillo: #FFD21E  
- Naranja: #FF9D00  
- Gris: #6B7280

---

*Empoderando a la comunidad de IA — un commit a la vez.*
```

'```markdown\n# Hugging Face: Construyendo el Futuro de la IA Juntos\n\n---\n\n## Sobre Hugging Face\n\nHugging Face está en el corazón de la revolución de la IA, sirviendo como la principal plataforma de colaboración para la comunidad de aprendizaje automático. Empoderamos a ingenieros, científicos y desarrolladores de aprendizaje automático en todo el mundo para compartir, descubrir y construir soluciones de IA abiertas y éticas. Nuestra misión es clara: **democratizar el aprendizaje automático de calidad — un commit a la vez.**\n\nEl Hugging Face Hub es un ecosistema vibrante donde las personas pueden alojar y colaborar en modelos ilimitados, conjuntos de datos y aplicaciones, acelerando la innovación en IA a través de la comunidad y herramientas de código abierto.\n\n---\n\n## Lo Que Ofrecemos\n\n### Modelos y Conjuntos de Datos  \n- Navega **más de 1 millón de modelos** que incluyen modelos avanzados de PLN, visión, audio y multimodalidad.  \n- Accede a **más de 250,000 conjuntos 

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant and fast-growing AI and machine learning community dedicated to advancing the future of open and ethical AI. It serves as a collaboration platform where machine learning engineers, scientists, and enthusiasts from around the world create, discover, and share machine learning models, datasets, and applications.

The Hugging Face Hub is the central place for hosting and collaborating on over **1 million models**, **250k+ datasets**, and **400k+ applications** spanning multiple modalities including text, images, video, audio, and 3D.

---

## What Hugging Face Offers

- **Models:** Browse and explore a vast collection of AI models, updated frequently by a diverse community. Popular trending models include image generators, video editors, and language models.
  
- **Datasets:** Access a rich catalog of datasets ranging from natural language prompts to persona datasets and beyond, enabling rapid model training and experimentation.
  
- **Spaces:** Deploy, share, and run AI applications on Hugging Face's platform, including innovative tools such as video generation from images, AI toolkits, and interactive applications.

- **Compute:** Easily accelerate your machine learning workloads with optimized inference endpoints and GPU-backed applications starting as low as $0.60/hour.

- **Enterprise Solutions:** Hugging Face provides tailored enterprise offerings to help organizations harness AI technology securely and effectively at scale.

---

## Company Culture

Hugging Face fosters an **open, collaborative, and ethical AI community** — empowering developers and researchers to work together transparently and inclusively. The culture emphasizes:

- **Community-driven innovation:** Encouraging open-source contributions and sharing knowledge.
- **Ethical AI development:** Committed to building responsible AI systems for the benefit of all.
- **Diversity and inclusion:** Supporting a broad and global community in building AI tools and resources.
- **Learning and growth:** Offering a platform for users to build their machine learning portfolios and grow their expertise.

---

## Customers & Community

Hugging Face serves a global community that includes:

- Machine learning researchers and practitioners
- Developers building innovative AI applications
- Enterprises seeking scalable AI solutions
- AI enthusiasts and learners who want to experiment and grow their skills

With over a million models and hundreds of thousands of datasets and applications, the platform connects users with cutting-edge AI technologies from companies like Tencent, NVIDIA, Facebook, and OpenAI — showcasing the power of open collaboration.

---

## Careers at Hugging Face

Interested in joining Hugging Face? The company values talented individuals passionate about AI, open-source technologies, and community building. Opportunities include roles in:

- Machine learning research & engineering
- Software development
- Cloud infrastructure and compute optimization
- Community management and developer relations
- Enterprise solutions & sales

At Hugging Face, you will work at the forefront of AI innovation while contributing to a global, open, and ethical AI movement.

---

## Get Involved

- **Explore AI models and datasets:** Visit [huggingface.co](https://huggingface.co) to browse millions of resources.
- **Build and share your own models and apps:** Create your portfolio in an open, collaborative environment.
- **Accelerate your projects:** Utilize Hugging Face’s Compute and Enterprise services.
- **Join the community:** Engage with thousands of practitioners working toward the future of AI.

---

### Contact & Follow

- Website: [https://huggingface.co](https://huggingface.co)
- Join the community to learn, collaborate, and innovate in AI!

---

*Hugging Face — Empowering the next generation of machine learning for a better, open future.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>